In [8]:
import torch
import torch.nn as nn
import torchvision.models as models

def get_pretrained_regression_model(output_size):
    # Load the pretrained ResNet18 model
    pretrained_model = models.resnet18(weights='DEFAULT')
    
    # Modify the last fully connected layer for regression with custom output size
    in_features = pretrained_model.fc.in_features
    pretrained_model.fc = nn.Linear(in_features, output_size)
    
    return pretrained_model

model = get_pretrained_regression_model(100)

In [67]:
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA 
from torchvision import transforms
from PIL import Image
import numpy as np

# Define an imaginary dataset class
class ImaginaryDataset(Dataset):
    def __init__(self, num_samples, transform=None, PCA = None):
        self.num_samples = num_samples
        self.transform = transform
        self.PCA = PCA
        self.data, self.output = self.generate_data()

    def generate_data(self):
        data = []
        output_concat = []
        for _ in range(self.num_samples):
            # Generate random image data (3 channels, 128x128 pixels)
            image = np.random.randint(0, 256, size=(3, 128, 128), dtype=np.uint8)
            image = Image.fromarray(np.transpose(image, (1, 2, 0)))

            # Generate random output array of length 1000
            output = np.random.rand(1000)

            data.append((image, output))
            output_concat.append(output)
        if self.PCA: self.PCA.fit(output_concat)
        return data, output_concat
    
    def give_output(self):
        return self.output
    
    def internal_PCA(self):
        if self.PCA:
            return self.PCA
        else:
            print(f'PCA is {self.PCA}')

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image, output = self.data[idx]

        if self.transform:
            image = self.transform(image)
        if self.PCA:
            output = self.PCA.transform(output.reshape(1, -1))
        return image, torch.FloatTensor(output[0])

# Define data transformations (you can customize these as needed)
data_transform = transforms.Compose([
    transforms.ToTensor(),
])


# Create an instance of the ImaginaryDataset
num_samples = 1000  # You can adjust this based on your needs
imaginary_dataset = ImaginaryDataset(num_samples, transform= data_transform)


# Create a DataLoader for batch processing
# batch_size = 32
# data_loader = DataLoader(imaginary_dataset, batch_size=batch_size, shuffle=True)

In [62]:
class ImaginaryDataset(Dataset):
    def __init__(self, images_list, outputs_list, transform=None, PCA=None):
        self.num_samples = len(images_list)
        self.transform = transform
        self.PCA = PCA
        self.data, self.output = self.load_data(images_list, outputs_list)

    def load_data(self, images_list, outputs_list):
        data = []
        output_concat = []

        for i in range(self.num_samples):
            # Load image from the given list
            image = Image.fromarray(images_list[i])

            # Load output array from the given list
            output = outputs_list[i]

            data.append((image, output))
            output_concat.append(output)

        if self.PCA:
            self.PCA.fit(output_concat)

        return data, output_concat

    def give_output(self):
        return self.output

    def internal_PCA(self):
        if self.PCA:
            return self.PCA
        else:
            print(f'PCA is {self.PCA}')

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image, output = self.data[idx]

        if self.transform:
            image = self.transform(image)
        if self.PCA:
            output = self.PCA.transform(output.reshape(1, -1))

        return image, torch.FloatTensor(output[0])

tensor([-0.6295, -1.0589, -0.0295,  0.1368, -0.0329,  0.6135, -0.6347, -1.0426,
        -0.6552,  0.2255,  0.7754, -0.7037, -0.3348,  0.0709, -0.6169,  0.2675,
         0.2397,  0.6179,  0.8363,  0.8152, -0.6182, -0.9547, -0.7746, -0.2705,
         0.5320, -0.2302,  0.1947, -0.8367,  0.2591,  0.0209,  0.6226,  0.2442,
        -0.2488,  0.1452,  0.1646,  0.6997,  1.0926, -0.5920, -0.2921, -0.0297,
         0.2005,  0.2934, -0.5573,  0.1135,  0.9171,  0.1379, -0.7114,  0.1247,
        -0.4917,  0.4868,  0.2106,  0.7978, -0.3085, -0.4237,  0.0493,  1.1658,
        -0.4858,  0.0470, -0.6473,  0.6047,  0.5868,  0.3117, -0.2822, -0.4220,
        -0.3039,  0.0859,  0.1376,  0.4752,  0.9518,  0.0762,  0.3508, -0.0720,
         0.4628, -0.2626,  0.4634, -0.1505, -0.0965, -0.1135, -0.3443, -0.1266,
        -0.1408,  0.6696,  0.0095,  0.7941, -0.3037,  0.3988,  0.2276, -0.1188,
        -0.4072,  0.0746, -0.3886, -0.2150,  0.5003,  0.1679, -0.0914, -0.2793,
         0.2941, -0.0484,  0.2021,  0.41

In [68]:
Frozen_PCA = imaginary_dataset.internal_PCA()

PCA is None


In [65]:
len(Frozen_PCA.inverse_transform(imaginary_dataset[0][1]))

1000